In [1]:
# 8.5.1_replacement_analysis_finite_horizon.ipynb
""" 8.5.1 Opitmal Replacement Planning under finite study period """
import numpy_financial as npf
from EngFinancialPy import Asset, pprint_list

In [2]:
marr = 0.1

In [3]:
# Defender data
MV0_d = 5000
MV_d = [4000, 3000, 2000, 1000 ]  
E_d = [5500, 6600, 7800, 8800 ]
defender = Asset(MV0_d, MV_d, E_d, marr, name="Defender")

In [4]:
# Challenger data
MV0_c = 20000
MV_c = [15000, 11250, 8500, 6500,  4750 ]  
E_c = [ 2000,  3000, 4620, 8000, 12000 ]
challenger = Asset(MV0_c, MV_c, E_c, marr, age=0, name="Challenger")

In [5]:
# Check defender's useful life TC values
life_d = defender.useful_life()
print(f"Defender remaining life = {life_d} years")
TC_d = defender.TC()
pprint_list("Defender TC", TC_d)

Defender remaining life = 4 years
Defender TC = 7,000.00   8,000.00   9,100.00   10,000.00   


In [6]:
# Check challenger's useful life and TC values
life_c = challenger.useful_life()
print(f"Challenger useful life = {life_c} years")
TC_c = challenger.TC()
pprint_list("Challenger TC", TC_c)

Challenger useful life = 5 years
Challenger TC = 9,000.00   8,250.00   8,495.00   10,850.00   14,400.00   


In [7]:
# Perform Replacement Analysis under finite planning horizon
print("\nReplacement Analysis under finitie planning horizon")
N = 6  # study period
print("Study period = 6 years, assume challenger is repeatable")
# Generate feasible replacement plans
plans = [(k1,k2,k3) for k1 in range(life_d+1) for k2 in range(life_c+1)
             for k3 in range(life_c+1) if (k1+k2+k3==N) and (k2>0 or k3==0)]
print(f"Number of feasible plans = {len(plans)}")


Replacement Analysis under finitie planning horizon
Study period = 6 years, assume challenger is repeatable
Number of feasible plans = 19


In [8]:
# Compute the Equivalent Present Cost of each plan
EPC_dt = {}
TC_CF_dt = {}
for k1,k2,k3 in plans:
    TC_CF = TC_d[0:k1] + TC_c[0:k2] + TC_c[0:k3]
    EPC = npf.npv(marr, [0]+TC_CF)
    EPC_dt[(k1,k2,k3)] = EPC
    TC_CF_dt[(k1,k2,k3)] = TC_CF
    
print("Plan, TC Cash Flows, EPC Table:")
for i, p in enumerate(plans):
    print(f"{i+1:3d}. {p}:", "".join(f"{x:7,.0f}" for x in TC_CF_dt[p]), f" ={EPC_dt[p]:10,.2f}")

Plan, TC Cash Flows, EPC Table:
  1. (0, 1, 5):   9,000  9,000  8,250  8,495 10,850 14,400  = 42,485.80
  2. (0, 2, 4):   9,000  8,250  9,000  8,250  8,495 10,850  = 38,795.96
  3. (0, 3, 3):   9,000  8,250  8,495  9,000  8,250  8,495  = 37,447.35
  4. (0, 4, 2):   9,000  8,250  8,495 10,850  9,000  8,250  = 39,038.32
  5. (0, 5, 1):   9,000  8,250  8,495 10,850 14,400  9,000  = 42,814.65
  6. (1, 1, 4):   7,000  9,000  9,000  8,250  8,495 10,850  = 37,597.62
  7. (1, 2, 3):   7,000  9,000  8,250  9,000  8,250  8,495  = 36,064.93
  8. (1, 3, 2):   7,000  9,000  8,250  8,495  9,000  8,250  = 36,047.40
  9. (1, 4, 1):   7,000  9,000  8,250  8,495 10,850  9,000  = 37,619.46
 10. (1, 5, 0):   7,000  9,000  8,250  8,495 10,850 14,400  = 40,667.62
 11. (2, 1, 3):   7,000  8,000  9,000  9,000  8,250  8,495  = 35,801.97
 12. (2, 2, 2):   7,000  8,000  9,000  8,250  9,000  8,250  = 35,617.10
 13. (2, 3, 1):   7,000  8,000  9,000  8,250  8,495  9,000  = 35,726.89
 14. (2, 4, 0):   7,000  8,000  

In [9]:
# Find optimal replacement plan
best_plan, min_EPC = min(EPC_dt.items(), key=lambda x: x[1])
print(f"\nOptimal plan = {best_plan}")
print(f"Min EPC (opportunity cost) = {min_EPC:,.2f}")


Optimal plan = (2, 2, 2)
Min EPC (opportunity cost) = 35,617.10


In [10]:
# Compute EUAC over study period under cash flow approach
EUAC_cf = -npf.pmt(marr, N, min_EPC-MV0_d, 0)
print(f"Optimal EUAC (cash flow) = {EUAC_cf:,.2f}")

Optimal EUAC (cash flow) = 7,029.91
